In [56]:
# Read sequences from sequences.fasta

import pandas as pd
import numpy as np
from Bio import SeqIO
from Bio import pairwise2

# Read sequences from sequences.fasta
sequences = []
for record in SeqIO.parse("sequences.fasta", "fasta"):
    sequences.append(record.seq)
    
def calculate_distance(seqs):
    table = np.zeros((len(seqs), len(seqs)))
    for i, seq1 in enumerate(seqs):
        for j, seq2 in enumerate(seqs):
            real_score = pairwise2.align.globalxx(seq1, seq2, score_only=True)
            shuffled_seq1 = ''.join(np.random.choice(list(seq1), len(seq1)))
            shuffled_seq2 = ''.join(np.random.choice(list(seq2), len(seq2)))
            shuf_score = pairwise2.align.globalxx(shuffled_seq1, shuffled_seq2, score_only=True)
            real_scoreii = pairwise2.align.globalxx(seq1, seq1, score_only=True)
            real_scorejj = pairwise2.align.globalxx(seq2, seq2, score_only=True)
            effect_score = -(np.log((real_score - shuf_score)/((1/2)*(real_scoreii + real_scorejj) - shuf_score)))
            table[j][i] = effect_score
            table[i][j] = effect_score
    return table
    
table = calculate_distance(sequences)
print(pd.DataFrame(table))

    

          0         1         2         3         4         5         6  \
0 -0.000000  0.392406  0.629151  0.334892  0.363543  0.662858  0.349674   
1  0.392406 -0.000000  0.832909  0.418670  0.489872  0.962276  0.461594   
2  0.629151  0.832909 -0.000000  0.796005  0.658361  0.301668  0.586192   
3  0.334892  0.418670  0.796005 -0.000000  0.413687  0.826214  0.416823   
4  0.363543  0.489872  0.658361  0.413687 -0.000000  0.629159  0.321267   
5  0.662858  0.962276  0.301668  0.826214  0.629159 -0.000000  0.632589   
6  0.349674  0.461594  0.586192  0.416823  0.321267  0.632589 -0.000000   
7  1.653149  1.486220  1.955603  1.548350  1.505069  1.962259  1.461018   
8  0.853233  0.874476  1.413106  0.764099  0.867917  1.370413  0.892241   

          7         8  
0  1.653149  0.853233  
1  1.486220  0.874476  
2  1.955603  1.413106  
3  1.548350  0.764099  
4  1.505069  0.867917  
5  1.962259  1.370413  
6  1.461018  0.892241  
7 -0.000000  1.408056  
8  1.408056 -0.000000  


In [26]:
table = [
    [10],
    [2, 10],
    [10, 6, 10],
    [8, 10, 8, 10],
    [8, 10, 8, 10, 4]
]

In [24]:
table = [
    [0.397327],
    [0.650773, 0.850239],
    [0.337268, 0.420779, 0.812019],
    [0.363543, 0.487295, 0.668837, 0.409534],
    [0.683444, 0.898138, 0.289359, 0.837510, 0.638963],
]

In [27]:
def lowest_cell(table):
    # Set default to infinity
    min_cell = float("inf")
    x, y = -1, -1

    for i in range(len(table)):
        for j in range(len(table[i])):
            if table[i][j] < min_cell:
                min_cell = table[i][j]
                x, y = i, j

    return x, y

def join_labels(labels, a, b):
    if b < a:
        a, b = b, a
    print(labels, a, b)
    labels[a] = "(" + labels[a] + "," + labels[b] + ")"

    del labels[b]


def join_table(table, a, b):
    if b < a:
        a, b = b, a

    row = []
    for i in range(0, a):
        row.append((table[a][i] + table[b][i])/2)
    table[a] = row
    
    for i in range(a+1, b):
        table[i][a] = (table[i][a]+table[b][i])/2
        
    for i in range(b+1, len(table)):
        table[i][a] = (table[i][a]+table[i][b])/2
        del table[i][b]

    del table[b]

def UPGMA(table, labels):
    while len(labels) > 1:
        x, y = lowest_cell(table)

        join_table(table, x, y)

        join_labels(labels, x, y)

    return labels[0]


def alpha_labels(start, end):
    labels = []
    for i in range(ord(start), ord(end)+1):
        labels.append(chr(i))
    return labels

labels = alpha_labels("A", "F")
UPGMA(table, labels)



['A', 'B', 'C', 'D', 'E', 'F'] 0 1
['(A,B)', 'C', 'D', 'E', 'F'] 3 3
['(A,B)', 'C', 'D', 'F'] 0 1
['((A,B),C)', 'D', 'F'] 0 1


IndexError: list index out of range